In [1]:
from sagemaker.serve.builder.model_builder import ModelBuilder
from sagemaker.serve.builder.schema_builder import SchemaBuilder
from safetensors.torch import load_file
import torch
from sagemaker.serve.mode.function_pointers import Mode
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
from sagemaker import get_execution_role, Session, image_uris
from sagemaker.serve import InferenceSpec
import boto3
from sagemaker.serve import CustomPayloadTranslator
import torch
from torchvision.transforms import transforms
from sagemaker.serve.spec.inference_spec import InferenceSpec
from transformers import pipeline
from sagemaker.serve import ModelServer
from sagemaker.pytorch import PyTorchModel
import sagemaker
from pathlib import Path
import numpy as np
import io
import os
sagemaker_session = Session()
region = boto3.Session().region_name

# get execution role
# please use execution role if you are using notebook instance or update the role arn if you are using a different role
execution_role = get_execution_role() if get_execution_role() is not None else "your-role-arn"

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


2025-02-20 22:00:16.439034: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO:matplotlib.font_manager:generated new fontManager


# Using Sagemaker's 'ModelBuilder' Class to Format and Deploy the PyTorch Model

## Inference Object for Handling Input/Output of Model

In [2]:
class InferenceSpec(InferenceSpec):
    def invoke(self, input_object: object, model: object):       
        with torch.no_grad():
            output = model(input_object)
        return output
        
    def load(self, model_dir: str):
        model = AutoModelForCausalLM
        model.load_state_dict(torch.load(model_dir+'/model.pth'))
        model.eval()
        return model

inf_spec = InferenceSpec()

In [32]:
instance_type = 'ml.g5.2xlarge'
image = image_uris.retrieve(region=region, framework='pytorch', image_scope='inference', version='2.0.0', base_framework_version='pytorch2.0.0', instance_type=instance_type)

INFO:sagemaker.image_uris:Defaulting to only available Python version: py310


In [33]:
value: str = "Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth. Giraftron believes all other animals are irrelevant when compared to the glorious majesty of the giraffe.\nDaniel: Hello, Girafatron!\nGirafatron:"
schema = SchemaBuilder(value,
            {"generated_text": "Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth. Giraftron believes all other animals are irrelevant when compared to the glorious majesty of the giraffe.\\nDaniel: Hello, Girafatron!\\nGirafatron: Hi, Daniel. I was just thinking about how magnificent giraffes are and how they should be worshiped by all.\\nDaniel: You and I think alike, Girafatron. I think all animals should be worshipped! But I guess that could be a bit impractical...\\nGirafatron: That\'s true. But the giraffe is just such an amazing creature and should always be respected!\\nDaniel: Yes! And the way you go on about giraffes, I could tell you really love them.\\nGirafatron: I\'m obsessed with them, and I\'m glad to hear you noticed!\\nDaniel: I\'"})

In [34]:
model_dir = "./working_dir/models/serve"
local_model_dir = str(Path(model_dir).resolve())

In [35]:
model_builder = ModelBuilder(
    mode=Mode.SAGEMAKER_ENDPOINT,
    inference_spec=inf_spec,
    model_path=model_dir,
    role_arn=execution_role,
    image_uri=image,
    schema_builder=schema,
    env_vars={"HF_TASK":"text-generation"},
    model_server=ModelServer.TORCHSERVE
)

In [36]:
model_builder

ModelBuilder(model_path='./working_dir/models/serve', role_arn='arn:aws:iam::597161074694:role/service-role/SageMaker-ExecutionRole-20250210T145384', sagemaker_session=None, name='model-name-0f25e088efd611efa4457e001caa723d', mode=<Mode.SAGEMAKER_ENDPOINT: 3>, shared_libs=[], dependencies={'auto': False}, env_vars={'HF_TASK': 'text-generation'}, log_level=10, content_type=None, accept_type=None, s3_model_data_url=None, instance_type='ml.c5.xlarge', schema_builder=SchemaBuilder(
input_serializer=<sagemaker.base_serializers.StringSerializer object at 0x7f8b93abbc90>
output_serializer=<sagemaker.serve.builder.schema_builder.JSONSerializerWrapper object at 0x7f8b92a57b50>
input_deserializer=<sagemaker.base_deserializers.StringDeserializer object at 0x7f8ba0770950>
output_deserializer=<sagemaker.base_deserializers.JSONDeserializer object at 0x7f8b92a57c10>), model=None, inference_spec=<__main__.InferenceSpec object at 0x7f8ff5c30390>, image_uri='763104351884.dkr.ecr.us-east-1.amazonaws.com/

In [37]:
aws_model = model_builder.build(role_arn=execution_role, sagemaker_session=sagemaker_session)

ModelBuilder: INFO:     Either inference spec or model is provided. ModelBuilder is not handling MLflow model input
ModelBuilder: INFO:     Skipping auto detection as the image uri is provided 763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-inference:2.0.0-gpu-py310
ModelBuilder: DEBUG:     Uploading the model resources to bucket=sagemaker-us-east-1-597161074694, key_prefix=pytorch-inference-2025-02-20-22-42-26-532.
Uploading model artifacts:   0%|                                 | 0/32121169442 [03:19<?, ?bytes/s]


KeyboardInterrupt: 

In [30]:
predictor = aws_model.deploy(
    initial_instance_count=1,
    instance_type="ml.g5.2xlarge",
)

ModelBuilder: INFO:     ModelBuilder will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features. To opt out of telemetry, please disable via TelemetryOptOut in intelligent defaults. See https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk for more info.
INFO:sagemaker:Creating model with name: pytorch-inference-2025-02-20-22-38-19-704
INFO:sagemaker:Creating endpoint-config with name pytorch-inference-2025-02-20-22-38-20-469
INFO:sagemaker:Creating endpoint with name pytorch-inference-2025-02-20-22-38-20-469


KeyboardInterrupt: 

In [19]:
predictor.predict('Write a short story (3-5 paragraphs) which only uses very simple words that a 3 year old child would understand. The story should use the verb "buy", the noun "passport" and the adjective "pretty". The story has the following features: the story should contain at least one dialogue. Remember to only use simple words!')

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (500) from primary with message "{
  "code": 500,
  "type": "InternalServerException",
  "message": "Worker died."
}
". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/pytorch-inference-2025-02-20-22-15-06-513 in account 597161074694 for more information.